In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras
from time import time
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import os, shutil, glob



In [2]:
#Prepare directories with images on classified folders

images = []
df = pd.read_csv('HAM10000_metadata.csv')
df_dx = df.dx.unique()

if os.path.exists('data_images'):
    shutil.rmtree('data_images') 

os.mkdir('data_images')

    
for dx in df_dx:
    dirName = 'data_images/' + dx
    if not os.path.exists(dirName):
        os.mkdir(dirName)
        
images_1 = [f for f in listdir('HAM10000_images_part_1') if isfile(join('HAM10000_images_part_1', f))]
images_2 = [f for f in listdir('HAM10000_images_part_2') if isfile(join('HAM10000_images_part_2', f))]

for f in images_1:
    dx_dir = 'data_images/' + df.loc[df['image_id'] == os.path.splitext(f)[0], 'dx'].values.item()
    shutil.copy('HAM10000_images_part_1/' + f, dx_dir)
    images.append(dx_dir + '/' + f)
    
for f in images_2:
    dx_dir = 'data_images/' + df.loc[df['image_id'] == os.path.splitext(f)[0], 'dx'].values.item()
    shutil.copy('HAM10000_images_part_2/' + f, dx_dir)
    images.append(dx_dir + '/' + f)
    
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [3]:
#Generate more samples by mirroring images

#import cv2, csv

#last_sample_name = 34321

#if os.path.exists('HAM10000_metadata_with_more_samples.csv'):
#    os.remove('HAM10000_metadata_with_more_samples.csv')
    
#shutil.copy('HAM10000_metadata.csv', 'HAM10000_metadata_with_more_samples.csv')

#def flipImage(img, image_path, direction, last_sample_name, csv_line):
#    flipped = cv2.flip(img, direction)
#    cv2.imwrite(image_path, flipped)
#    writeLinesInCsv('HAM10000_metadata_with_more_samples.csv', last_sample_name, csv_line)

#def writeLinesInCsv(file, last_sample_name, csv_line):
#    with open(file, 'a') as fd:
#        csv_line[1] = 'ISIC_00' + str(last_sample_name)
#        fd.write(csv_line[0] + ',' + csv_line[1] + ',' + csv_line[2] + ',' + csv_line[3] + ',' + csv_line[4] + ',' + csv_line[5] + ',' + csv_line[6] +'\n')

#for image in images:
        
#    with open('HAM10000_metadata_with_more_samples.csv') as f_obj:
#        reader = csv.reader(f_obj, delimiter=',')
#        for line in reader:
#            if image.split('/')[2].replace('.jpg', '') in line:  
#                csv_line = line
#                break
    
#    image_path = image.replace(image.split('/')[2], '') + 'ISIC_00' + str(last_sample_name) + '.jpg'
#    flipImage(cv2.imread(image), image_path, 0, last_sample_name, csv_line) # Vertical
#    last_sample_name += 1
    
#    image_path = image.replace(image.split('/')[2], '') + 'ISIC_00' + str(last_sample_name) + '.jpg'
#    flipImage(cv2.imread(image), image_path, 1, last_sample_name, csv_line) # Horizontal
#    last_sample_name += 1               

In [4]:
# Separate a validation dataset in other folder moving 30% images. (¿BALANCEAR DATOS?)
import random

if os.path.exists('data_test'):
    shutil.rmtree('data_test') 

os.mkdir('data_test')
list = os.listdir('data_images/')
for folder in list:
    os.mkdir('data_test/' + folder)
    folder_list = os.listdir('data_images/' + folder)
    for to_copy in random.sample(folder_list, int(len(folder_list)*.3)):
        shutil.move('data_images/' + folder + '/' + to_copy, 'data_test/' + folder + '/' + to_copy)

In [20]:
batch_size = 20
train_data_dir = "C:/Users/fiamm/Documents/Master_SIANI/CI/Cayetano/TrabajoFinal/data_images"
validation_data_dir = "C:/Users/fiamm/Documents/Master_SIANI/CI/Cayetano/TrabajoFinal/data_test"

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=315,
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(validation_data_dir,
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

Found 7014 images belonging to 7 classes.
Found 3001 images belonging to 7 classes.


In [24]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
         activation='relu',
         input_shape=(400,400,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss = tensorflow.keras.losses.categorical_crossentropy,
              optimizer = tensorflow.keras.optimizers.Adadelta(),
              metrics = ['accuracy'])

In [25]:
epochs = 200

es = EarlyStopping(monitor='val_accuracy', 
                   mode='max', 
                   verbose=1, 
                   patience=6, 
                   restore_best_weights=True)

model.fit_generator(train_generator,
                    epochs = epochs,
                    validation_data = validation_generator,
                    callbacks = [es])

Epoch 1/200
351/351 [==============================] - 1536s 4s/step - loss: 1.2553 - accuracy: 0.6458 - val_loss: 1.0691 - val_accuracy: 0.6701
Epoch 2/200
351/351 [==============================] - 1502s 4s/step - loss: 1.1141 - accuracy: 0.6654 - val_loss: 1.0259 - val_accuracy: 0.6701
Epoch 3/200
351/351 [==============================] - 1496s 4s/step - loss: 1.0309 - accuracy: 0.6813 - val_loss: 1.0071 - val_accuracy: 0.6701
Epoch 4/200
351/351 [==============================] - 1566s 4s/step - loss: 1.0594 - accuracy: 0.6741 - val_loss: 0.9957 - val_accuracy: 0.6701
Epoch 5/200
351/351 [==============================] - 1506s 4s/step - loss: 1.0414 - accuracy: 0.6717 - val_loss: 0.9894 - val_accuracy: 0.6701
Epoch 6/200
351/351 [==============================] - 1503s 4s/step - loss: 1.0206 - accuracy: 0.6719 - val_loss: 0.9809 - val_accuracy: 0.6701
Epoch 7/200
351/351 [==============================] - 1514s 4s/step - loss: 1.0165 - accuracy: 0.6676 - val_loss: 0.9824 - val_ac

In [26]:
model.save("modelos/modeloIntento4.h5")